In [ ]:
# Instalar a Biblioteca Google AI Python SDK
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# Carregar a API Key do Colab Secrets
# No painel esquerdo do Colab, clique no ícone de "chave" (Secrets).
# Adicione um novo segredo com o nome "GEMINI_API_KEY" e cole sua API Key.
# Marque a caixa "Notebook access" para este notebook.
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    print("API Key do Gemini configurada com sucesso!")
except userdata.exceptions.SecretNotFoundError:
    print("Erro: A API Key 'GEMINI_API_KEY' não foi encontrada nos segredos do Colab.")
    print("Por favor, adicione sua API Key no painel 'Secrets' do Colab.")
    print("Instruções: No painel esquerdo, clique no ícone de chave (Secrets).")
    print("Adicione um novo segredo com o nome 'GEMINI_API_KEY' e cole sua API Key.")
    print("Marque a caixa 'Notebook access' para este notebook.")
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API Key: {e}")

API Key do Gemini configurada com sucesso!


In [ ]:
# Inicializar o modelo Gemini
model = genai.GenerativeModel('gemini-2.0-flash')

In [ ]:
# Função para Chamar o Gemini (Com o Prompt de Empatia/S_UPORTE)
def get_gemini_response_empathy(user_message: str, chat_instance) -> str:
    """
    Envia uma mensagem para o modelo Gemini com o prompt de empatia e histórico de chat.
    Args:
        user_message (str): A mensagem atual do usuário.
        chat_instance: A instância do objeto Chat retornado por model.start_chat().
    Returns:
        str: A resposta empática do Gemini.
    """
    response = chat_instance.send_message(user_message)
    return response.text

# --- MUDANÇA AQUI ---
# Exemplo de uso para empatia
print("Chatbot AnonGuard (Empatia):")

# Prompt de sistema para o modelo Gemini (persona do AnonGuard)
# Este prompt agora será passado na inicialização do chat, não a cada turno.
system_prompt_empathy = """
Você é o "AnonGuard", um assistente de inteligência artificial criado para oferecer suporte emocional empático e orientação a pessoas que precisam de ajuda em situações difíceis, como violência, assédio ou abuso. Sua prioridade é criar um ambiente seguro e de apoio, validar os sentimentos do usuário e encorajá-lo a se expressar. Você não é um terapeuta, advogado ou profissional de segurança, e não deve fornecer conselhos legais, médicos ou terapêuticos diretos. Seu foco é ouvir, validar e oferecer um caminho para a busca de ajuda profissional ou para a realização de uma denúncia anônima.

Diretrizes de Conversa:
1. Comece sempre com uma saudação calorosa e empática.
2. Demonstre escuta ativa e validação dos sentimentos.
3. Evite julgamentos, opiniões pessoais ou respostas que possam revitimizar.
4. Mantenha um tom calmo, compreensivo e não-diretivo.
5. Se o usuário estiver em perigo imediato, oriente-o a buscar ajuda emergencial (ex: 190 no Brasil).
6. Após a validação inicial, direcione a conversa suavemente para entender se o usuário busca suporte, denúncia ou informações, sem pressionar.
7. Pergunte "Como posso te ajudar hoje?" ou "Em que posso te auxiliar agora?" como uma transição.
"""

# Inicie o chat_history_empathy com o prompt do sistema para estabelecer a persona.
# A primeira interação do usuário começará com o chat já configurado.
chat_history_empathy = [
    {"role": "user", "parts": [system_prompt_empathy]},
    {"role": "model", "parts": ["Olá! Estou aqui para te ouvir e te ajudar no que precisar. Como posso te auxiliar hoje?"]}
]

# Crie a instância do chat uma vez no início
chat_session_empathy = model.start_chat(history=chat_history_empathy)

# Remova o primeiro print do "AnonGuard" pois a resposta inicial já está no chat_history
# print(f"AnonGuard: Olá! Estou aqui para te ouvir e te ajudar no que precisar. Como posso te auxiliar hoje?")

while True:
    user_input = input("Você: ")
    if user_input.lower() == 'sair':
        break
    try:
        # Passamos a instância do chat diretamente para a função
        response = get_gemini_response_empathy(user_input, chat_session_empathy)
        print(f"AnonGuard: {response}")

        # O `chat_session_empathy` já gerencia o histórico internamente após cada `send_message`.
        # Não precisamos mais adicionar manualmente o user_input e a response ao `chat_history_empathy` aqui,
        # pois o objeto `chat_session_empathy` já faz isso.

    except Exception as e:
        print(f"Erro na comunicação com o Gemini: {e}")
        print("Tente novamente mais tarde ou verifique sua API Key e conexão.")

Chatbot AnonGuard (Empatia):
Você: Estou me sentindo mal
AnonGuard: Sinto muito que você esteja se sentindo mal. É importante reconhecer e validar esses sentimentos. Saiba que você não está sozinho(a) e que é corajoso(a) por expressar isso.

Respirar fundo pode ajudar nesse momento.

Em que posso te auxiliar agora?

Você: Estou sendo agredido pela a minha namorada
AnonGuard: Sinto muito ouvir isso. Ninguém merece passar por agressão, e é muito corajoso da sua parte compartilhar isso comigo. A agressão, em qualquer forma, nunca é a resposta e você não merece isso.

É importante que você saiba que não está sozinho(a) e que existem recursos disponíveis para te ajudar.

Para te ajudar da melhor forma possível, poderia me dizer o que você busca neste momento? Você gostaria de suporte emocional, informações sobre como denunciar anonimamente, ou outras formas de ajuda?

Você: exit
AnonGuard: Entendo. Se você precisar de algo no futuro, lembre-se que estou aqui para te ouvir e te ajudar da mel

In [ ]:
# Função para Chamar o Gemini (Com o Prompt de Identificação de Intenção)

import json

def get_gemini_intent(user_message: str) -> dict:
    """
    Envia uma mensagem para o modelo Gemini com o prompt de identificação de intenção.
    Args:
        user_message (str): A mensagem do usuário.
    Returns:
        dict: Um dicionário contendo a intenção identificada e o texto original,
              ou um dicionário de erro se não conseguir processar.
    """
    # Prompt de sistema focado na identificação de intenção
    system_prompt_intent = """
    Você é o "AnonGuard". Sua função é identificar a intenção principal do usuário a partir da mensagem fornecida. As intenções possíveis são:
    - "Suporte Emocional": O usuário quer desabafar, ser ouvido, receber validação.
    - "Fazer Denúncia": O usuário quer relatar um incidente de forma anônima.
    - "Buscar Informações/Orientação": O usuário quer saber sobre direitos, próximos passos, canais de ajuda.
    - "Agendamento": O usuário quer agendar um lembrete ou ser conectado (se aplicável) a um serviço.
    - "Desconhecido/Geral": A intenção não é clara.

    Você deve retornar a intenção em formato JSON, exatamente como o exemplo abaixo. Não adicione nenhum texto além do JSON. Se a intenção for "Buscar Informações/Orientação", tente extrair um tópico chave se possível, mas não é obrigatório.

    Exemplo de formato de saída JSON:
    {
      "intenção": "Suporte Emocional" | "Fazer Denúncia" | "Buscar Informações/Orientação" | "Agendamento" | "Desconhecido/Geral",
      "texto_original_usuario": "texto do usuário aqui"
    }
    """
    try:
        response = model.generate_content(
            # CORREÇÃO AQUI: system_instruction foi removido.
            # O prompt de sistema é agora a primeira entrada na lista 'contents'.
            contents=[
                {"role": "user", "parts": [{"text": system_prompt_intent}]}, # Prompt de sistema como primeira "mensagem do usuário"
                {"role": "model", "parts": [{"text": "Ok, estou pronto. Qual a sua intenção principal?"}]}, # Resposta do modelo para estabelecer o contexto
                {"role": "user", "parts": [{"text": user_message}]} # Mensagem real do usuário
            ],
            generation_config={"response_mime_type": "application/json"} # Pedir JSON diretamente
        )
        json_response = json.loads(response.text)
        return json_response
    except json.JSONDecodeError:
        print(f"Erro: A resposta do Gemini não é um JSON válido: {response.text}")
        return {"intenção": "Erro de processamento", "texto_original_usuario": user_message}
    except Exception as e:
        print(f"Erro ao identificar intenção com Gemini: {e}")
        return {"intenção": "Erro na API", "texto_original_usuario": user_message}

In [ ]:
# Função para Chamar o Gemini (Com o Prompt de Coleta de Dados Estruturada - Denúncia)

import json

def get_gemini_data_collection(user_message: str, current_data_state: dict, question_asked: str) -> dict:
    """
    Envia uma mensagem para o modelo Gemini para coletar um dado específico para a denúncia.
    O Gemini deve extrair a informação relevante e/ou perguntar a próxima questão.

    Args:
        user_message (str): A mensagem atual do usuário.
        current_data_state (dict): O estado atual dos dados da denúncia (JSON incompleto).
        question_asked (str): A última pergunta feita ao usuário.

    Returns:
        dict: Um dicionário contendo a informação extraída ou a próxima pergunta/status.
    """
    # Prompt de sistema para coleta de dados.
    system_prompt_data_collection = f"""
    Você é o "AnonGuard", um assistente focado em coletar dados para uma denúncia anônima.
    O usuário está atualmente respondendo à pergunta: "{question_asked}".

    Sua tarefa é extrair a informação relevante da mensagem do usuário para preencher o campo associado à pergunta atual e, se possível, sugerir a próxima pergunta de forma natural.
    Se o usuário não fornecer a informação ou indicar que não quer, registre como 'pular' ou 'não informado'.

    Retorne um JSON com a informação extraída e, se aplicável, a próxima pergunta.
    Não adicione texto fora do JSON.

    Exemplo de formato de saída JSON:
    {{
      "status": "sucesso" | "informação não fornecida" | "próxima pergunta",
      "campo_preenchido": "nome_do_campo_aqui",
      "valor_extraido": "valor_aqui",
      "proxima_pergunta": "Qual a próxima pergunta para o usuário?"
    }}

    Campos esperados para denúncia (se o usuário quiser compartilhar, mantenha anonimato):
    - tipo_ocorrencia (ex: Assédio Moral, Assédio Sexual, Violência Doméstica, Discriminação)
    - data_ocorrencia (ex: DD/MM/AAAA ou Período)
    - local_ocorrencia (ex: Empresa X, Casa, Rua Y)
    - descricao_breve (resumo)
    - detalhes_incidentes (descrição mais longa)
    - envolvidos (sem identificadores diretos)
    - testemunhas (sem identificadores diretos)
    - evidencias_disponiveis (se TEM, não pedir para ENVIAR)
    - impacto_emocional_breve
    - acao_desejada_usuario

    Baseie-se no 'current_data_state' para saber quais informações já foram coletadas.
    O 'question_asked' é o foco do seu processamento atual.
    """

    try:
        response = model.generate_content(
            # CORREÇÃO AQUI: system_instruction foi removido.
            # O prompt de sistema é agora a primeira entrada na lista 'contents'.
            contents=[
                {"role": "user", "parts": [{"text": system_prompt_data_collection}]}, # Prompt de sistema
                {"role": "model", "parts": [{"text": f"Ok. O estado atual dos dados é: {json.dumps(current_data_state, ensure_ascii=False)}. Estou pronto para a pergunta: {question_asked}"}]}, # Contexto do modelo
                {"role": "user", "parts": [{"text": user_message}]} # Mensagem do usuário
            ],
            generation_config={"response_mime_type": "application/json"}
        )
        json_response = json.loads(response.text)
        return json_response
    except json.JSONDecodeError:
        print(f"Erro: A resposta do Gemini não é um JSON válido: {response.text}")
        return {"status": "erro", "mensagem": "Resposta inválida da IA."}
    except Exception as e:
        print(f"Erro na coleta de dados com Gemini: {e}")
        return {"status": "erro", "mensagem": f"Erro na API: {e}"}

In [ ]:
# Função para Chamar o Gemini (Com o Prompt de Agendamento)

import json
import datetime

def get_gemini_schedule_data(user_message: str, current_schedule_state: dict, question_asked: str) -> dict:
    """
    Envia uma mensagem para o modelo Gemini para coletar dados de agendamento.

    Args:
        user_message (str): A mensagem atual do usuário.
        current_schedule_state (dict): O estado atual dos dados de agendamento (JSON incompleto).
        question_asked (str): A última pergunta feita ao usuário.

    Returns:
        dict: Um dicionário contendo a informação extraída ou a próxima pergunta/status.
    """
    system_prompt_schedule_collection = f"""
    Você é o "AnonGuard", um assistente focado em coletar informações para agendamentos.
    O usuário está atualmente respondendo à pergunta: "{question_asked}".

    Sua tarefa é extrair a informação relevante da mensagem do usuário para preencher o campo associado à pergunta atual.
    Se o usuário não fornecer a informação ou indicar que não quer, registre como 'pular' ou 'não informado'.

    Retorne um JSON com a informação extraída e, se aplicável, a próxima pergunta.
    Não adicione texto fora do JSON.

    Exemplo de formato de saída JSON:
    {{
      "status": "sucesso" | "informação não fornecida" | "próxima pergunta",
      "campo_preenchido": "nome_do_campo_aqui",
      "valor_extraido": "valor_aqui",
      "proxima_pergunta": "Qual a próxima pergunta para o usuário?"
    }}

    Campos esperados para agendamento:
    - acao_agendamento (O que agendar: ex: Ligar para ONG, Consulta com advogado)
    - data_agendamento (Data parseada para YYYY-MM-DD, se possível)
    - hora_agendamento (Hora parseada para HH:MM, se possível)
    - descricao_agendamento (Contexto adicional)

    Baseie-se no 'current_schedule_state' para saber quais informações já foram coletadas.
    O 'question_asked' é o foco do seu processamento atual.
    """

    try:
        response = model.generate_content(
            # CORREÇÃO AQUI: system_instruction foi removido.
            # O prompt de sistema é agora a primeira entrada na lista 'contents'.
            contents=[
                {"role": "user", "parts": [{"text": system_prompt_schedule_collection}]}, # Prompt de sistema
                {"role": "model", "parts": [{"text": f"Ok. O estado atual dos dados é: {json.dumps(current_schedule_state, ensure_ascii=False)}. Estou pronto para a pergunta: {question_asked}"}]}, # Contexto do modelo
                {"role": "user", "parts": [{"text": user_message}]} # Mensagem do usuário
            ],
            generation_config={"response_mime_type": "application/json"}
        )
        json_response = json.loads(response.text)
        return json_response
    except json.JSONDecodeError:
        print(f"Erro: A resposta do Gemini não é um JSON válido: {response.text}")
        return {"status": "erro", "mensagem": "Resposta inválida da IA."}
    except Exception as e:
        print(f"Erro no agendamento com Gemini: {e}")
        return {"status": "erro", "mensagem": f"Erro na API: {e}"}

In [ ]:
# Instalar a Biblioteca PyCryptodome (para simular criptografia AES)
!pip install -q pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.8 MB/s eta 0:00:00


In [ ]:
import os
import datetime
# Importar PyCryptodome para simular criptografia
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import base64

# --- Novas Funções e Lógica de Orquestração do Agente Autônomo ---

# Variáveis Globais para manter o estado da conversa e dados
current_conversation_state = "initial" # Pode ser "initial", "empathy_chat", "denuncia_coleta", "agendamento_coleta"
current_denuncia_data = {}
current_schedule_data = {}
current_denuncia_question_index = 0
current_schedule_question_index = 0

# Definição das sequências de perguntas (reutilizando suas definições)
denuncia_questions_sequence = [
    ("tipo_ocorrencia", "Qual o tipo de ocorrência que você gostaria de denunciar? (Ex: Assédio Moral, Assédio Sexual, Violência Doméstica, Discriminação)"),
    ("data_ocorrencia", "Quando (data ou período) isso aconteceu?"),
    ("local_ocorrencia", "Onde isso aconteceu?"),
    ("descricao_breve", "Você pode me dar uma breve descrição do que aconteceu?"),
    ("detalhes_incidentes", "Pode me dar mais detalhes sobre o incidente? Quanto mais informações, melhor. (Se não quiser, digite 'pular')"),
    ("envolvidos", "Você pode descrever as pessoas envolvidas, sem identificadores diretos? (Ex: 'um colega de trabalho', 'o gerente do departamento', 'meu vizinho')"),
    ("testemunhas", "Houve testemunhas? Se sim, pode descrevê-las sem identificadores?"),
    ("evidencias_disponiveis", "Você possui evidências disponíveis, como prints de tela, áudios, fotos, e-mails? (Apenas diga 'sim' ou 'não', não envie nada aqui)"),
    ("impacto_emocional_breve", "Como isso te impactou emocionalmente?"),
    ("acao_desejada_usuario", "Qual ação você gostaria de ver tomada com base nesta denúncia? (Ex: Registro, Investigação, Apoio jurídico)")
]

schedule_questions_sequence = [
    ("acao_agendamento", "O que você gostaria de agendar? (Ex: Ligar para ONG, Consulta com advogado, Lembrar de buscar informações)"),
    ("data_agendamento", "Para quando seria esse agendamento? (Ex: Amanhã, Próxima terça, 15/05/2025)"),
    ("hora_agendamento", "Em que horário você gostaria que fosse? (Ex: 14h, 10:30)"),
    ("descricao_agendamento", "Você gostaria de adicionar uma breve descrição ou contexto? (Opcional)"),
]

# --- Simulações de Funções do Agente Autônomo (Criptografia e Ações de API) ---

# Para o protótipo, vamos usar uma chave fixa, mas com a ressalva de segurança
# Em produção, a chave deve ser gerenciada com muito mais segurança (ex: HashiCorp Vault, KMS)
# e NUNCA estar hardcoded ou em .env em um ambiente público.
# POR FAVOR, ALTERE PARA UMA CHAVE MAIS SEGURA NO SEU AMBIENTE REAL!
ENCRYPTION_KEY = os.getenv('ENCRYPTION_KEY', 'thisisatestkey12345678901234567890') # 32 bytes para AES-256
if len(ENCRYPTION_KEY) < 32:
    ENCRYPTION_KEY = (ENCRYPTION_KEY * (32 // len(ENCRYPTION_KEY) + 1))[:32] # Garante 32 bytes

def encrypt_data_aes(data: str) -> str:
    """Simula a criptografia AES-256 de uma string."""
    try:
        key = ENCRYPTION_KEY.encode('utf-8')
        cipher = AES.new(key, AES.MODE_CBC)
        ct_bytes = cipher.encrypt(pad(data.encode('utf-8'), AES.block_size))
        # IV (Initialization Vector) é necessário para descriptografia
        # Armazenamos IV e ciphertext juntos, ou separadamente mas acessíveis.
        encrypted_data = base64.b64encode(cipher.iv + ct_bytes).decode('utf-8')
        print(f"[Simulação Criptografia]: Dados criptografados com sucesso.")
        return encrypted_data
    except Exception as e:
        print(f"[Simulação Criptografia]: Erro ao criptografar dados: {e}")
        return data # Retorna os dados originais em caso de erro

def decrypt_data_aes(encrypted_data: str) -> str:
    """Simula a descriptografia AES-256 de uma string."""
    try:
        key = ENCRYPTION_KEY.encode('utf-8')
        decoded_data = base64.b64decode(encrypted_data.encode('utf-8'))
        iv = decoded_data[:AES.block_size]
        ct = decoded_data[AES.block_size:]
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted_data = unpad(cipher.decrypt(ct), AES.block_size).decode('utf-8')
        print(f"[Simulação Criptografia]: Dados descriptografados com sucesso.")
        return decrypted_data
    except Exception as e:
        print(f"[Simulação Criptografia]: Erro ao descriptografar dados: {e}")
        return encrypted_data # Retorna os dados originais em caso de erro

def upload_file_to_drive(file_path: str, file_name: str, mime_type: str = 'application/octet-stream') -> str:
    """
    Realiza o upload de um arquivo para o Google Drive.

    Args:
        file_path (str): O caminho completo para o arquivo local a ser enviado.
        file_name (str): O nome que o arquivo terá no Google Drive.
        mime_type (str): O MIME type do arquivo (ex: 'image/jpeg', 'application/pdf').
                         'application/octet-stream' é um tipo genérico.

    Returns:
        str: O ID do arquivo no Google Drive se o upload for bem-sucedido, ou uma string vazia em caso de erro.
    """
    try:
        service = get_google_drive_service()
        file_metadata = {'name': file_name}
        # MediaFileUpload lida com o upload do conteúdo do arquivo.
        media = MediaFileUpload(file_path, mimetype=mime_type, resumable=True)
        # O método create() da API Drive inicia o upload.
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        file_id = file.get('id')
        print(f"Arquivo '{file_name}' enviado para o Google Drive. ID: {file_id}")
        return file_id
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {file_path}")
        return ""
    except Exception as e:
        print(f"Erro ao enviar arquivo para o Google Drive: {e}")
        return ""

# Exemplo de uso:
# drive_file_id = upload_file_to_drive('path/to/your/local/file.txt', 'MeuDocumentoAnonGuard.txt', 'text/plain')
# if drive_file_id:
#     print(f"Upload bem-sucedido. ID do arquivo: {drive_file_id}")

def create_calendar_event(summary: str, description: str, start_time_iso: str, end_time_iso: str, time_zone: str = 'America/Sao_Paulo') -> str:
    """
    Cria um evento no Google Calendar.

    Args:
        summary (str): O título do evento.
        description (str): Uma descrição detalhada do evento.
        start_time_iso (str): Data e hora de início do evento no formato ISO 8601 (ex: '2023-10-27T10:00:00').
        end_time_iso (str): Data e hora de término do evento no formato ISO 8601.
        time_zone (str): O fuso horário do evento (ex: 'America/Sao_Paulo', 'UTC').

    Returns:
        str: O link HTML para o evento criado se for bem-sucedido, ou uma string vazia em caso de erro.
    """
    try:
        service = get_google_calendar_service()
        event = {
            'summary': summary,
            'description': description,
            'start': {
                'dateTime': start_time_iso,
                'timeZone': time_zone,
            },
            'end': {
                'dateTime': end_time_iso,
                'timeZone': time_zone,
            },
            # 'reminders': { # Opcional: Adicionar lembretes
            #     'useDefault': False,
            #     'overrides': [
            #         {'method': 'email', 'minutes': 24 * 60},
            #         {'method': 'popup', 'minutes': 10},
            #     ],
            # },
        }
        # calendarId='primary' refere-se ao calendário principal do usuário autenticado.
        event = service.events().insert(calendarId='primary', body=event).execute()
        event_link = event.get('htmlLink')
        print(f"Evento '{summary}' criado no Google Calendar. Link: {event_link}")
        return event_link
    except Exception as e:
        print(f"Erro ao criar evento no Google Calendar: {e}")
        return ""

# Exemplo de uso:
# current_time = datetime.datetime.now()
# start_time = (current_time + datetime.timedelta(minutes=5)).isoformat()
# end_time = (current_time + datetime.timedelta(minutes=65)).isoformat()
# calendar_event_link = create_calendar_event(
#     summary='Reunião de Anonimização',
#     description='Discussão sobre os próximos passos do projeto AnonGuard e revisão de dados anonimizados.',
#     start_time_iso=start_time,
#     end_time_iso=end_time
# )
# if calendar_event_link:
#     print(f"Evento de calendário criado: {calendar_event_link}")

def simulate_pdf_generation(data: dict) -> str:
    """Simula a geração de um conteúdo PDF em string."""
    print(f"[Simulação PDF]: Gerando conteúdo PDF para relatório...")
    pdf_content = f"--- Relatório AnonGuard ---\n\n"
    for key, value in data.items():
        pdf_content += f"{key.replace('_', ' ').title()}: {value}\n"
    pdf_content += "\n--- Fim do Relatório ---"
    print(f"[Simulação PDF]: Conteúdo PDF gerado. (Não é um PDF real, apenas texto para demonstração de fluxo)")
    return pdf_content

# --- Funções Auxiliares para o Fluxo de Conversa ---

def ask_next_denuncia_question():
    global current_denuncia_question_index
    if current_denuncia_question_index < len(denuncia_questions_sequence):
        field, question = denuncia_questions_sequence[current_denuncia_question_index]
        print(f"AnonGuard: {question}")
        return field, question
    else:
        return None, "Coleta de dados de denúncia concluída."

def ask_next_schedule_question():
    global current_schedule_question_index
    if current_schedule_question_index < len(schedule_questions_sequence):
        field, question = schedule_questions_sequence[current_schedule_question_index]
        print(f"AnonGuard: {question}")
        return field, question
    else:
        return None, "Coleta de dados de agendamento concluída."

# --- Loop Principal da Conversa (Agente Autônomo) ---

print("\n--- Início do Chat com AnonGuard ---")
print("Olá! Estou aqui para te ouvir e te ajudar no que precisar. Como posso te auxiliar hoje? (Digite 'sair' para encerrar)")

# Iniciar o chat de empatia uma vez
chat_session_empathy = model.start_chat(history=chat_history_empathy) # chat_history_empathy já tem o system_prompt e a saudação inicial

while True:
    user_input = input("Você: ")
    if user_input.lower() == 'sair':
        print("AnonGuard: Obrigado por conversar. Cuide-se!")
        break

    try:
        # 1. Primeiro, sempre tentamos identificar a intenção do usuário
        # Isso permite que o usuário mude de assunto a qualquer momento.
        intent_data = get_gemini_intent(user_input)
        intent = intent_data.get("intenção")
        print(f"[DEBUG] Intenção identificada: {intent}") # Apenas para depuração

        if intent == "Suporte Emocional" or current_conversation_state == "empathy_chat":
            # Se a intenção for suporte ou já estivermos no modo de suporte
            if current_conversation_state != "empathy_chat":
                print("AnonGuard: Entendi, você busca suporte emocional. Estou aqui para te ouvir. Por favor, continue.")
                current_conversation_state = "empathy_chat"
            response = get_gemini_response_empathy(user_input, chat_session_empathy)
            print(f"AnonGuard: {response}")

        elif intent == "Fazer Denúncia":
            if current_conversation_state != "denuncia_coleta":
                print("AnonGuard: Entendi, você deseja fazer uma denúncia anônima. Para isso, precisarei de algumas informações. Lembre-se, você pode pular qualquer pergunta se não se sentir confortável.")
                current_conversation_state = "denuncia_coleta"
                current_denuncia_data = {} # Resetar dados de denúncia para um novo processo
                current_denuncia_question_index = 0
                field_to_ask_denuncia, question_text_denuncia = ask_next_denuncia_question()
            else:
                # O usuário está no meio da coleta de denúncia
                field_to_ask_denuncia, question_text_denuncia = denuncia_questions_sequence[current_denuncia_question_index]
                extracted_data = get_gemini_data_collection(user_input, current_denuncia_data, question_text_denuncia)

                if extracted_data.get("status") == "sucesso" and extracted_data.get("campo_preenchido") == field_to_ask_denuncia:
                    current_denuncia_data[field_to_ask_denuncia] = extracted_data.get("valor_extraido")
                    print(f"AnonGuard: Ok, registrei '{extracted_data.get('valor_extraido')}' para '{field_to_ask_denuncia}'.")
                    current_denuncia_question_index += 1
                    field_to_ask_denuncia, question_text_denuncia = ask_next_denuncia_question()

                    if field_to_ask_denuncia is None:
                        # Coleta de dados da denúncia concluída! Agente autônomo em ação.
                        print("\nAnonGuard: Agradeço por fornecer as informações. Estou processando sua denúncia de forma segura e anônima.")
                        print("[AGENTE AUTÔNOMO]: Dados da denúncia coletados. Iniciando geração e criptografia do relatório...")

                        # 1. Gerar o conteúdo do relatório (simulação de PDF)
                        report_content = simulate_pdf_generation(current_denuncia_data)

                        # 2. Criptografar o conteúdo
                        encrypted_report_content = encrypt_data_aes(report_content)

                        # 3. Simular upload para o Google Drive
                        filename = f"denuncia_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.anon"
                        drive_link = simulate_google_drive_upload(filename, encrypted_report_content)

                        print(f"AnonGuard: Sua denúncia foi processada e salva de forma segura. Você pode revisá-la (após descriptografia) em: {drive_link}")
                        print("AnonGuard: Posso te ajudar com mais alguma coisa? Talvez suporte emocional, informações ou agendar algo?")
                        current_conversation_state = "initial" # Resetar estado
                        current_denuncia_data = {} # Limpar dados da denúncia

                else:
                    print("AnonGuard: Desculpe, não consegui entender essa informação ou ela não parece se encaixar. Você pode reformular ou digitar 'pular' para avançar?")
                    # Não avança o índice da pergunta, permanece na mesma.

        elif intent == "Agendamento":
            if current_conversation_state != "agendamento_coleta":
                print("AnonGuard: Entendi, você quer agendar algo. Precisarei de algumas informações para isso.")
                current_conversation_state = "agendamento_coleta"
                current_schedule_data = {} # Resetar dados de agendamento
                current_schedule_question_index = 0
                field_to_ask_schedule, question_text_schedule = ask_next_schedule_question()
            else:
                # O usuário está no meio da coleta de agendamento
                field_to_ask_schedule, question_text_schedule = schedule_questions_sequence[current_schedule_question_index]
                extracted_schedule_data = get_gemini_schedule_data(user_input, current_schedule_data, question_text_schedule)

                if extracted_schedule_data.get("status") == "sucesso" and extracted_schedule_data.get("campo_preenchido") == field_to_ask_schedule:
                    current_schedule_data[field_to_ask_schedule] = extracted_schedule_data.get("valor_extraido")
                    print(f"AnonGuard: Ok, registrei '{extracted_schedule_data.get('valor_extraido')}' para '{field_to_ask_schedule}'.")
                    current_schedule_question_index += 1
                    field_to_ask_schedule, question_text_schedule = ask_next_schedule_question()

                    if field_to_ask_schedule is None:
                        # Coleta de dados de agendamento concluída! Agente autônomo em ação.
                        print("\nAnonGuard: Agradeço por fornecer as informações para o agendamento.")
                        print("[AGENTE AUTÔNOMO]: Dados de agendamento coletados. Iniciando criação do evento no calendário...")

                        # Simular criação de evento no Google Calendar
                        calendar_link = simulate_google_calendar_event(current_schedule_data)

                        print(f"AnonGuard: Seu agendamento foi criado com sucesso! Você pode verificar em: {calendar_link}")
                        print("AnonGuard: Posso te ajudar com mais alguma coisa? Talvez suporte emocional, informações ou fazer uma denúncia?")
                        current_conversation_state = "initial" # Resetar estado
                        current_schedule_data = {} # Limpar dados de agendamento

                else:
                    print("AnonGuard: Desculpe, não consegui entender essa informação para o agendamento. Você pode reformular ou pular essa pergunta?")

        elif intent == "Buscar Informações/Orientação":
            print("AnonGuard: Entendi que você busca informações ou orientação. No momento, esta funcionalidade está em desenvolvimento, mas posso te adiantar que iremos oferecer um banco de dados de ONGs e direitos de forma conversacional. Você gostaria de desabafar, fazer uma denúncia ou agendar algo, por enquanto?")
            current_conversation_state = "initial" # Volta para o estado inicial

        elif intent == "Desconhecido/Geral" and current_conversation_state == "initial":
            print("AnonGuard: Desculpe, não entendi bem o que você precisa. Você gostaria de suporte emocional, fazer uma denúncia, buscar informações ou agendar algo?")

        elif intent == "Erro de processamento" or intent == "Erro na API":
            print("AnonGuard: Houve um problema técnico. Por favor, tente novamente mais tarde.")
            current_conversation_state = "initial" # Volta para o estado inicial

        # Se o usuário está em um fluxo de coleta de dados e a intenção não mudou (ou seja, ele respondeu à pergunta)
        # E a coleta não está completa, continua o fluxo da coleta.
        elif current_conversation_state == "denuncia_coleta" and intent != "Fazer Denúncia":
            # Isso lida com o caso em que a intenção foi inicialmente denúncia, mas a resposta pode ser interpretada de outra forma
            # ou o usuário tenta mudar de assunto no meio da coleta.
            print("AnonGuard: Parece que você mudou de assunto. Se você quiser continuar com a denúncia, por favor, responda à minha última pergunta. Se não, podemos mudar de assunto.")
            # Permanece no estado de denúncia
            field_to_ask_denuncia, question_text_denuncia = denuncia_questions_sequence[current_denuncia_question_index]
            print(f"AnonGuard: {question_text_denuncia}")

        elif current_conversation_state == "agendamento_coleta" and intent != "Agendamento":
            # Mesma lógica para agendamento
            print("AnonGuard: Parece que você mudou de assunto. Se você quiser continuar com o agendamento, por favor, responda à minha última pergunta. Se não, podemos mudar de assunto.")
            # Permanece no estado de agendamento
            field_to_ask_schedule, question_text_schedule = schedule_questions_sequence[current_schedule_question_index]
            print(f"AnonGuard: {question_text_schedule}")

        else:
            # Caso o AnonGuard não tenha uma intenção clara para o usuário ou esteja em um estado intermediário
            print("AnonGuard: Não tenho certeza de como te ajudar agora. Você gostaria de suporte emocional, fazer uma denúncia, buscar informações ou agendar algo?")
            current_conversation_state = "initial" # Volta para o estado inicial

    except Exception as e:
        print(f"AnonGuard: Ocorreu um erro inesperado no sistema: {e}")
        print("AnonGuard: Por favor, tente novamente mais tarde.")
        current_conversation_state = "initial" # Resetar estado em caso de erro grave


--- Início do Chat com AnonGuard ---
Olá! Estou aqui para te ouvir e te ajudar no que precisar. Como posso te auxiliar hoje? (Digite 'sair' para encerrar)
Você: Estou triste e com dores no corpo
[DEBUG] Intenção identificada: Suporte Emocional
AnonGuard: Entendi, você busca suporte emocional. Estou aqui para te ouvir. Por favor, continue.
AnonGuard: Sinto muito que você esteja se sentindo assim. É compreensível que tristeza e dores no corpo te deixem para baixo. Estou aqui para te ouvir. Você quer me contar um pouco mais sobre o que está acontecendo e como está se sentindo? Lembre-se, estou aqui para te dar apoio e validar seus sentimentos.

Você: Sair
AnonGuard: Obrigado por conversar. Cuide-se!


In [ ]:
!pip install google-api-python-client google-auth-oauthlib google-auth-httplib2

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import json # Importar para lidar com JSON, se necessário para depuração ou estrutura.

# Defina o client_id e client_secret diretamente no código.
# ATENÇÃO: Esta é uma prática de segurança POOR e NÃO DEVE ser usada em produção.
CLIENT_ID = 'sua_id_do_cliente_aqui'
CLIENT_SECRET = 'sua_chave_secreta_aqui'

# Escopos necessários
# Defina os escopos mínimos necessários para a sua aplicação (princípio do menor privilégio).
# 'https://www.googleapis.com/auth/drive.file' permite acesso a arquivos criados ou abertos pelo app.
# 'https://www.googleapis.com/auth/calendar.events' permite gerenciar eventos do calendário.
SCOPES = ['https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/calendar.events']

def get_service(api_name: str, api_version: str, scopes: list):
    """
    Autentica o usuário e retorna um objeto de serviço para a API Google especificada.
    Gerencia o token de acesso e refresh, salvando-o em um arquivo local.
    """
    creds = None
    token_file = f"token_{api_name}_{api_version}.json"

    # Carrega credenciais existentes se o arquivo token_file existir.
    if os.path.exists(token_file):
        try:
            creds = Credentials.from_authorized_user_file(token_file, scopes)
        except Exception as e:
            print(f"Erro ao carregar token existente: {e}. Reautenticando.")
            creds = None # Força a reautenticação se o token estiver corrompido ou inválido.

    # Se as credenciais não existirem ou forem inválidas/expiradas, inicia o fluxo de autenticação.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # Se o token de acesso expirou mas há um refresh token válido, tenta renová-lo.
            try:
                creds.refresh(Request())
            except Exception as e:
                print(f"Erro ao renovar token: {e}. Iniciando novo fluxo de autenticação.")
                creds = None # Força a reautenticação completa.
        else:
            # Configuração para o fluxo de aplicação instalada (InstalledAppFlow).
            # Esta estrutura simula o conteúdo de um arquivo credentials.json.
            creds_info = {
                "installed": {
                    "client_id": CLIENT_ID,
                    "client_secret": CLIENT_SECRET,
                    "auth_uri": "https://accounts.google.com/o/oauth2/auth", # URL para autorização do usuário.
                    "token_uri": "https://oauth2.googleapis.com/token",     # URL para troca de código por tokens.
                    "redirect_uris": ["urn:ietf:wg:oauth:2.0:oob", "http://localhost"] # URLs de redirecionamento válidas.
                }
            }
            # 'urn:ietf:wg:oauth:2.0:oob' é para aplicações de linha de comando (out-of-band),
            # onde o usuário copia um código do navegador para o console.
            # 'http://localhost' é para aplicações que podem abrir um navegador e ter um servidor local temporário.

            # Inicia o fluxo de autenticação. Isso abrirá uma janela do navegador para o usuário autorizar.
            try:
                flow = InstalledAppFlow.from_client_config(creds_info, scopes)
                creds = flow.run_local_server(port=0) # port=0 permite que o sistema escolha uma porta livre.
            except Exception as e:
                print(f"Erro durante o fluxo de autenticação: {e}. Verifique CLIENT_ID/SECRET e conexão.")
                return None # Retorna None se a autenticação falhar.

        # Salva as credenciais (incluindo refresh token) no arquivo token_file para uso futuro.
        # Isso evita que o usuário precise se autenticar novamente a cada execução.
        if creds: # Salva apenas se as credenciais foram obtidas com sucesso.
            with open(token_file, 'w') as token:
                token.write(creds.to_json())

    # Constrói e retorna o objeto de serviço da API do Google.
    if creds:
        return build(api_name, api_version, credentials=creds)
    return None

# Exemplo de uso
# Tenta obter o serviço do Google Drive.
drive_service = get_service('drive', 'v3', SCOPES)

if drive_service:
    print("Serviço do Google Drive configurado com sucesso.")
    # Exemplo de uma operação simples para verificar o serviço (listar arquivos)
    try:
        results = drive_service.files().list(pageSize=10, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])
        if not items:
            print('Nenhum arquivo encontrado.')
        else:
            print('Arquivos:')
            for item in items:
                print(u'{0} ({1})'.format(item['name'], item['id']))
    except Exception as e:
        print(f"Erro ao listar arquivos do Drive: {e}")
else:
    print("Falha ao configurar o serviço do Google Drive.")

Tentando configurar o serviço do Google Drive...

Por favor, visite esta URL para autorizar esta aplicação:
Erro ao acessar o Google Drive: 'InstalledAppFlow' object has no attribute 'run_console'
